In [7]:
from experiment.utils import dbutils, transformation
from experiment.api import label_studio

In [8]:
db = dbutils.DatabaseUtils()

In [9]:
# get reports directly from database
query = """
            SELECT
                report_id,
                patient_no,
                protocol_no,
                report_original,
                report_prompted as text,
                report_length,
                patient_report_count
            FROM
                annotation.upload_tasks
            WHERE
                report_id NOT IN (
                SELECT
                    (DATA ->> 'report_id')::INT AS report_id
                FROM
                    public.task)
                AND report_prompted != ''
        """

# get values from the database
df_upload_tasks = db.read_sql_query(query)

# output tasks as a csv file
output_path = (
    transformation.get_project_root() / "tmp" / "data" / "upload_tasks.csv"
)
df_upload_tasks.to_csv(output_path, index=False)
df_upload_tasks

,report_id,patient_no,protocol_no,report_original,text,report_length,patient_report_count
0,26,2004004697,22023602,RAPOR TARİHİ:27/12/2021 FİLM NO:\n\nKontrast...,REPORT DATE: 27/12/2021 FILM NO: Contrastles...,128,19
1,16913,2008509284,22344550,RAPOR TARİHİ: 21/02/2022 FİLM NO:12506694\n\...,REPORT DATE: 02/21/2022 FILM NO:12506694 Co...,165,20
2,16909,2008509284,22344550,RAPOR TARİHİ:08/02/2022 FİLM NO:12506525\n\n...,REPORT DATE: 08/02/2022 FILM NO: 12506525 Con...,113,20


In [10]:
# upload tasks to label studio
label_studio.upload_csv_tasks(csv_path=output_path, project_id=7)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

{"task_count":3,"annotation_count":0,"prediction_count":0,"duration":0.057959794998168945,"file_upload_ids":[32],"could_be_tasks_list":true,"found_formats":{".csv":1},"data_columns":["patient_report_count","patient_no","report_id","report_length","text","protocol_no","report_original"]}

100  6438  100   287  100  6151    850  18237 --:--:-- --:--:-- --:--:-- 19275


In [11]:
label_studio.stop_label_studio()